In [1]:
from skimage.color import rgb2gray
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import ndimage
from matplotlib import image as mpimg
import imageio

In [2]:
#   PROCEDURE: CREATE_SOBEL_FILTER() OUTPUT: SOBEL_COMBINED
def create_sobel_filter(strong_edge =3, weak_edge= 2, low_threshold=0.001):
    #create three arrays for the vertical sobel filter called 'top','middle','bottom'
    strong_edge_neg = strong_edge * -1
    weak_edge_neg = weak_edge * -1
    low_threshold_neg = low_threshold * -1
    
    top_v = np.array([weak_edge_neg, low_threshold , weak_edge])
    middle_v = np.array([strong_edge_neg, 0 , strong_edge])
    bottom_v = np.array([weak_edge_neg, low_threshold , weak_edge])

    #create three arrays for the horizontal sobel filter called 'top','middle','bottom'
    top_h = np.array([weak_edge_neg, strong_edge_neg , weak_edge_neg])
    middle_h = np.array([low_threshold, 0 , low_threshold])
    bottom_h = np.array([weak_edge, strong_edge , weak_edge])

    #define the sobel filter for horizontal edges
    sobel_horizontal = np.array([top_h, middle_h, bottom_h])

    #define the sobel filter for vertical edges
    sobel_vertical = np.array([top_v, middle_v, bottom_v])

    #display the horizontal sobel filter
    print(sobel_horizontal)

    #display the vertical sobel filter
    print(sobel_vertical)

    #return the sobel filters
    return sobel_horizontal, sobel_vertical


In [3]:
#   PROCEDURE: APPLY_SOBEL_FILTERS(IMG, SH, SV) OUTPUT: SOBEL_COMBINED *IMG*

def apply_sobel_filters(img, sobel_horizontal, sobel_vertical):
    #apply the horizontal sobel filter to the image
    horizontal = ndimage.convolve(img, sobel_horizontal, mode='reflect')

    #apply the vertical sobel filter to the image
    vertical = ndimage.convolve(img, sobel_vertical, mode='reflect')

    #combine the horizontal and vertical sobel filters
    sobel_combined = np.hypot(horizontal, vertical)

    #display the result
    plt.imshow(sobel_combined, cmap='gray')

    #return the result
    return sobel_combined

In [4]:
#   PROCEDURE: IMPORT_PIC(PATH)  OUTPUT: GRAY, IMG

def import_pic(path):
    img = mpimg.imread(path)
    if len(img.shape) != 3:
        img = np.dstack((img, img, img))
    gray = rgb2gray(img)
    plt.imshow(gray, cmap='gray')
    return gray, img

In [5]:
#   PROCEDURE: CONVERT TO BINARY(IMG)  OUTPUT: BINARY
# Convert the sobel_combined image to a binary image
def convert_to_binary(img):
    #create a copy of the image
    binary = np.copy(img)

    #set the threshold for the binary image
    binary[img > 0.9999] = 1
    binary[img <= 0.9999] = 0

    #display the binary image
    plt.imshow(binary, cmap='gray')

    #return the binary image
    return binary

In [6]:
#        PROCEDURE: CHECK_IF_LINE(IMG, H, V, LENGTH = 10, THRESHOLD = 1.5) OUTPUT:'H'/'V'/'F'
#REMEMBER : IMG[VERT FROM TOP, HORIZ FROM LEFT]

def check_if_line(img, h, v, length = 10, threshold = 1.5):

    if h == 1 or v == 1 or h == img.shape[1]-1 or v == img.shape[0]-1:
       # print('error: on edge')
        return 0
    
    elif img[v,h] == 0:
       # print('error: pixel value @ point ' + str(v) + str(h) + ' = 0')
        return 0
    
    else:
        hmax = img.shape[1]
        vmax = img.shape[0]
        vmin = 0
        hmin = 0

        #check how much room on each side
        left = h-1
        right = hmax - h
        top = v-1
        bottom = vmax - v

        #define helpers
        if left > length:
            lefth = length
        else:
            lefth = left
       # print('lefth: ' + str(lefth))
        
        if right > length:
            righth = length
        else:
            righth = right
       # print('righth: ' + str(righth))

        if top > length:
            toph = length
        else:
            toph = top
       # print('toph: ' + str(toph))

        if bottom > length:
            bottomh = length = 10
        else:
            bottomh = bottom
       # print('bottomh: ' + str(bottomh))

        #check for vertical line
        #measure line above
        above = img[ v-toph:v , h]
       # print('above: ' +  str(above))
        line_above = sum(above)/len(above)
       # print('line_above: ' + str(line_above))

        #measure line above to the right
        above_right = img[v-toph:v, h+1]
       # print('above_right: ' + str(above_right))
        line_above_right = sum(above_right)/len(above_right)
       # print('line_above_right: ' + str(line_above_right))

        #measure line above to the left
        above_left = img[v-toph:v, h-1]
       # print('above_left: ' + str(above_left))
        line_above_left = sum(above_left)/len(above_left)
       # print('line_above_left: ' + str(line_above_left))

        #measure line below
        below = img[ v : v+bottomh, h]
       # print('below: ' + str(below))
        line_below = sum(below)/len(below)
       # print('line_below: ' + str(line_below))

        #measure line below to the right
        below_right = img[v:v+bottomh, h+1]
       # print('below_right: ' + str(below_right))
        line_below_right = sum(below_right)/len(below_right)
       # print('line_below_right: ' + str(line_below_right))

        #measure line below to the left
        below_left = img[v:v+bottomh, h-1]
       # print('below_left: ' + str(below_left))
        line_below_left = sum(below_left)/len(below_left)
       # print('line_below_left: ' + str(line_below_left))

        #check for horizontal line
        #measure horizontal line to the left
        before = img[ v, h-lefth : h]
       # print('before: ' + str(before))
        line_before = sum(before)/len(before)
       # print('line_before: ' + str(line_before))
        #measure horizontal line to the left up one
        before_up = img[ v-1, h-lefth : h]
       # print('before_up: ' + str(before_up))
        line_before_up = sum(before_up)/len(before_up)
       # print('line_before_up: ' + str(line_before_up))
        #measure horizontal line to the left down one
        before_down = img[ v+1, h-lefth : h]
       # print('before_down: ' + str(before_down))
        line_before_down = sum(before_down)/len(before_down)
       # print('line_before_down: ' + str(line_before_down))

        #measure horizontal line to the right
        after = img[ v, h : h+righth]
       # print('after: ' + str(after))
        line_after = sum(after)/len(after)
       # print('line_after: ' + str(line_after))
        #measure horizontal line to the right up one
        after_up = img[ v-1, h : h+righth]
       # print('after_up: ' + str(after_up))
        line_after_up = sum(after_up)/len(after_up)
       # print('line_after_up: ' + str(line_after_up))
        #measure horizontal line to the right down one
        after_down = img[ v+1, h : h+righth]
       # print('after_down: ' + str(after_down))
        line_after_down = sum(after_down)/len(after_down)
       # print('line_after_down: ' + str(line_after_down))

        #create scores
        scores = {}
        v_score = line_above + line_below
        vcr_score = line_above + line_below_right
        vcl_score = line_above + line_below_left
        vrc_score = line_above_right + line_below
        vlc_score = line_above_left + line_below
        vrl_score = line_above_right + line_below_left
        vlr_score = line_above_left + line_below_right

        h_score = line_before + line_after
        hcu_score = line_before + line_after_up
        hcd_score = line_before + line_after_down
        huc_score = line_before_up + line_after
        hcd_score = line_before_down + line_after
        hud_score = line_before_up + line_after_down
        hdu_score = line_before_down + line_after_up

        #add v_score to scores dictionary with key 'v'
        scores['v'] = v_score
        scores['vcr'] = vcr_score
        scores['vcl'] = vcl_score
        scores['vrc'] = vrc_score
        scores['vlc'] = vlc_score
        scores['vrl'] = vrl_score
        scores['vlr'] = vlr_score
        scores['h'] = h_score
        scores['hcu'] = hcu_score
        scores['huc'] = huc_score  
        scores['hcd'] = hcd_score
        scores['hud'] = hud_score
        scores['hdu'] = hdu_score

        #find max score
        max_score = 1.5
        h_tally = 0
        v_tally = 0
        max_scores = []
        for score in scores:
            if scores[score] > max_score:
                max_scores.append(score)
                if 'v' in score:
                    v_tally += 1
                elif 'h' in score:
                    h_tally += 1
        if h_tally > v_tally:
            return 'h'
        elif v_tally > h_tally:
            return 'v'
        else:
            return 'f'


In [7]:
#   PROCEDURE: MEASURE_GAP_BETWEEN_LINES(IMG,ENDING_CORD_V = , ENDING_CORD_H = , THRESHOLD = ) OUTPUT: DISTANCES, POINTS_W_GAPS
#
def measure_between_lines(img, starting_cord_v = 0, starting_cord_h = 0 , ending_cord_v=40, ending_cord_h= 40, threshold = 6):

    distances = []
    points_w_gaps = []
 
    #create sample
    sample = img[ starting_cord_v : ending_cord_v , starting_cord_h : ending_cord_h]
    #sample = img.copy()

    #iterate through sample
    #print('checkpoint 1')
    for i in range(1, sample.shape[0]-1):
        #print('checkpoint 2')
        for j in range(1, sample.shape[1]-1):
            #print('point in question: [', i,',', j, ']')
            #print('checkpoint 3 ')
            if sample[i,j] == 1 and sample [i,j+1] == 0 or sample[i,j] == 1 and sample[i+1,j] == 0:  
                #test first non-zero pixel adjacent to zero pixel
                ntest_point = [i,j]
                #print('#####test point 1: ', ntest_point)
                test_result1 = check_if_line(sample, ntest_point[1], ntest_point[0])
                #print('test result 1: ', test_result1)
                
                #for vertical line starting point
                #print('checkpoint 4')
                if test_result1 != 'f' and sample[ntest_point[0], ntest_point[1]+1] == 0:
               # if test_result1 == 'v': #or test_result1 == 'vcr' or test_result1 == 'vcl' or test_result1 == 'vrc' or test_result1 == 'vrl' or test_result1 == 'vlr':
                    #print('Vertical starting point approved')
                    j += 1
                    n = 1
                    tv = False
                    #print('checkpoint 5')
                    for k in range(j, j+threshold-1):
                        #print('checkpoint 6')
                        while tv == False:
                            n += 1
                            #print('checkpoint 7')
                            while n <= threshold and k < sample.shape[1]-1:
                                #print('n: ', n)
                                #print('k: ', k)
                                #print('checkpoint 8')
                                if sample[i,k] == 1 and sample[i, k-1] == 0:    
                                    ntest_point2 = [i,k]
                                    #print('~~~~~test point 2: ', ntest_point2)
                                    test_result2 = check_if_line(sample, ntest_point2[1], ntest_point2[0])
                                    #print('test result 2: ', test_result2)
                                    #print('checkpoint 9')
                                    if test_result2 != 'f':
                                   # if test_result2 == 'v' or test_result2 == 'vcr' or test_result2 == 'vcl' or test_result2 == 'vrc' or test_result2 == 'vrl' or test_result2 == 'vlr':
                                        #print("Vertical end point approved")
                                        distance = k-j
                                        distances.append(distance)
                                        #print('distance: ', distance)
                                        points_w_gaps.append([ntest_point, ntest_point2])
                                        #print('points with gaps: ', points_w_gaps)
                                        tv = True
                                        #print('tv: ', tv)
                                        #print('break 1')
                                        break

                                    else:
                                        k += 1
                                        #print('else1.1')
                                        #print('tv: ', tv)
                                        
                                else:
                                    #print('else1.2')
                                    #print('tv: ', tv)
                                    k += 1        
                                    #print('break 2')
                                    break
                            if n == threshold:
                                #print('break 3')
                                break
                        #print('break 4')
                        break 

                #for horizontal line starting point
                elif test_result1 != 'f' and sample[ntest_point[0]+1, ntest_point[1]] == 0:
               # elif test_result1 == 'h':# or test_result1 == 'hcu' or test_result1 == 'hcd' or test_result1 == 'huc' or test_result1 == 'hud' or test_result1 == 'hdu' and sample[ntest_point[0]+1, ntest_point[1]] == 0:
                    #print('checkpoint 10')
                    #print('ntestpoint[0]+1 : ', sample[ntest_point[0]+1, ntest_point[1]])
                    #print('Horizontal starting point approved')
                    sukme = i +1
                    n = 1
                    tv = False
                    #print('checkpoint 11')
                    for k in range(sukme, sukme+threshold-1):
                        #print('checkpoint 12')
                        while tv == False:
                            n += 1
                            #print('checkpoint 13')
                            while n <= threshold and k < sample.shape[0]-1:  
                                #print('n: ', n)
                                #print('sukme: ', k)
                                #print('checkpoint 14')
                                if sample[k, j] == 1 and sample[k-1, j] == 0:
                                    ntest_point2 = [k,j]
                                    #print('~~~~~~~~test point 2: ', ntest_point2)
                                    test_result2 = check_if_line(sample, ntest_point[1], ntest_point[0])
                                    #print('test result 2: ', test_result2)
                                    #print('checkpoint 15')
                                    if test_result2 != 'f':
                                   # if test_result2 == 'h' or test_result2 == 'hcu' or test_result2 == 'hcd' or test_result2 == 'huc' or test_result2 == 'hud' or test_result2 == 'hdu':
                                        #print("Horizontal end point approved")
                                        distance = k-sukme
                                        distances.append(distance)
                                        #print('distance: ', distance)
                                        points_w_gaps.append([ntest_point, ntest_point2])
                                        #print('points with gaps: ', points_w_gaps)
                                        tv = True
                                        #print('tv: ', tv)
                                        #print('break 5')
                                        break

                                    else:
                                        k += 1
                                        #print('else2.1')
                                        #print('tv: ', tv)

                                else:
                                    k += 1
                                    #print('else2.2')
                                    #print('tv: ', tv)
                                    #print('break 6')
                                    break
                                
                                
                            if n == threshold:
                                #print('break 7')
                                break
                        #print('break 8')
                        break    
                        
                else:
                    #print('else3')
                    #return to line 13
                    continue

            else:
                #print('else4')
                continue


    return distances, points_w_gaps#sample

In [8]:
#   PROCEDURE: FILL_BETWEEN_LINES(IMG, SET) OUTPUT: FILLED_IN
def fill_between_lines(img, set):
    #create a copy of image for drawing and returning
    filled_in = img.copy()
    
    #check if set is empty
    if len(set) == 0:
        print('set is empty')
    
    #check if img is single channel
    if len(img.shape) == 2:
        print('image is single channel')
    #iterate through set isolating each coordinate pair and drawing between them
    for i in range(len(set)-1):
        #print( 'i: ', i,)
        start_point = set[i][0]
        end_point = set[i][1]
        #print('starting point: ', start_point, 'ending point: ', end_point)

        starting_x = start_point[1]
        starting_y = start_point[0]
        ending_x = end_point[1]
        ending_y = end_point[0]
        #print('starting x: ', starting_x, 'ending x: ', ending_x, '\n','starting y: ', starting_y, 'ending y: ', ending_y)
        
        #see if line in vertical or horizontal
        if starting_x == ending_x:
            #print('vertical line between: ', starting_y, 'and', ending_y)
            for n in range(starting_y + 1, ending_y):
                filled_in[n, starting_x] = 1

        elif starting_y == ending_y:
            #print('horizontal line between: ', starting_x, 'and', ending_x)
            for k in range(starting_x + 1, ending_x):
                filled_in[starting_y, k] = 1

    return filled_in


In [9]:
# PROCEDURE: INPAINT(IMAGE, BINARY) OUTPUT: INPAINTED IMAGE
def inpaint(image, binary):
    #convert image to 8 bit 1 channel
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = np.uint8(image)
    #convert binary image to 8 bit 1 channel
    binary = np.uint8(binary)
    #inpaint the image
    dst = cv2.inpaint(image, binary, 3, cv2.INPAINT_NS)

    return dst

In [10]:
#   PROCEDURE: EXPORT(IMG, NAME, EP)  OUTPUT: BOOLEAN

def export(img, name, ep):
    path = ep + '/' + name + '.jpg'
    cv2.imwrite(path, img)
    
    #check if image was sucessfully written
    if os.path.isfile(path):
       # print('Image written to: ' + path)
        return True
    else:
       # print('Image not written to: ' + path)
        return False

In [11]:
#    PROCEDURE: PIPELINE(PATH, NAME, EP) OUTPUT: INPAINTED IMAGE

def pipeline(path, name, ep):
    
    gray, img = import_pic(path)
    sh, sv = create_sobel_filter()
    applied = apply_sobel_filters(gray, sh, sv)
    binary_mask = convert_to_binary(applied)
    mes, point_set = measure_between_lines(binary_mask, ending_cord_v = binary_mask.shape[0]-1, ending_cord_h= binary_mask.shape[1]-1)
    filled1 = fill_between_lines(binary_mask, point_set)
    mes2, point_set2 = measure_between_lines(filled1, ending_cord_v = filled1.shape[0]-1, ending_cord_h= filled1.shape[1]-1)
    filled2 = fill_between_lines(filled1, point_set2)
    inpainted = inpaint(img, filled2)
    exported = export(inpainted, name, ep)
    
    plt.imshow(inpainted, cmap='gray')
    plt.axis('off')
    plt.show()

    if exported == True:
        print('Image exported successfully as :' + name + ".jpg \n ~~to the folder: " + ep)
    else:
        print('Export failed, please check the path and try again')
#    return inpainted

In [1]:
#   PROCEDURE: ITER_PIP(PATH, EP, NUM, NAME = '') OUTPUT: NONE
def iter_pip(path, ep, num, name = ''):
    i = 0
    while i < num:
        img_path = path + name + '/' + str(i) + '.jpg'
        if os.path.isfile(img_path):
           print('isfile')
           pipeline(img_path, name + str(i), ep)
        else:
            print('isnotfile')
            pass
        i += 1

In [ ]:
iter_pip('ip',
         'ep',
         num)